### Import libraries 

In [1]:
import numpy as np
import pandas as pd

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, PrintfTickFormatter, Title

output_notebook()

Loading BokehJS ...

### Our goal 

<img src="../../images/figure_3.png"  width="50%" height="50%">

#### Colors 

In [2]:
GRAY1, GRAY2, GRAY3 = '#231F20', '#414040', '#555655'
GRAY4, GRAY5, GRAY6 = '#646369', '#76787B', '#828282'
GRAY7, GRAY8, GRAY9 = '#929497', '#A6A6A5', '#BFBEBE'
BLUE1, BLUE2, BLUE3, BLUE4 = '#174A7E', '#4A81BF', '#94B2D7', '#94AFC5'
RED1, RED2 = '#C3514E', '#E6BAB7'
GREEN1, GREEN2 = '#0C8040', '#9ABB59'
ORANGE1 = '#F79747'

#### Font 

In [3]:
FONT = 'Arial'

#### Preprocessing 

In [4]:
dataset = pd.read_csv('../../data/data_survey_pilot_program.csv')
dataset.shape

(5, 5)

In [5]:
dataset.head()

,Interest,Before,After,Formatted Before,Formatted After
0,Bored,11,12,11%,12%
1,Not great,5,6,5%,6%
2,OK,40,14,40%,14%
3,Kind of interested,25,30,25%,30%
4,Excited,19,38,19%,38%


In [6]:
dataset_hbar = dataset[['Interest', 'Before', 'After']].set_index('Interest').T.rename_axis('When').sort_values('When', ascending=True).reset_index()

In [7]:
dataset_hbar.head()

Interest,When,Bored,Not great,OK,Kind of interested,Excited
0,After,12,6,14,30,38
1,Before,11,5,40,25,19


In [8]:
dataset_hbar.columns.name = None
dataset_hbar['When'] = dataset_hbar['When'].apply(lambda x: x.upper())

In [9]:
dataset_hbar

,When,Bored,Not great,OK,Kind of interested,Excited
0,AFTER,12,6,14,30,38
1,BEFORE,11,5,40,25,19


In [10]:
# Bokeh doesn't allow you to add off-axis text. This is a little trick to do it
# This code adds extra space Above
data = []

new_row = {'When':' ', 'Bored':np.nan, 'Not great':np.nan, 'OK':np.nan, 'Kind of interested':np.nan, 'Excited':np.nan}
data.insert(0, new_row)

data = pd.DataFrame(data)[dataset_hbar.columns]
dataset_hbar = pd.concat([pd.DataFrame(data), dataset_hbar], ignore_index=True, axis=0)

# This code adds extra space Below
new_row = {'When':''}
dataset_hbar = dataset_hbar.append(new_row, ignore_index=True)

#### Plot 

In [11]:
# Set the source of the plot
source = ColumnDataSource(dataset_hbar)

interested = dataset['Interest'].to_list()
when = dataset_hbar['When'].to_list()


# Set color by category
colors = [GRAY6, GRAY6, ORANGE1, BLUE2, BLUE2]


# Create the figure
p = figure(x_range=(-17, 105),
           x_axis_location='above',
           y_range=when,
           plot_height=450,
           plot_width=750, 
           title='Pilot program was a success' + ' ' * 45,
           toolbar_location='below')


# Add bars to the figure
p.hbar_stack(interested, 
             y='When',             
             color=colors,
             height=0.65, 
             line_width=1.2, 
             line_color='white',
             source=source)


# Add subtitles
p.add_layout(Title(text="How do you feel about science?", 
                   align="left",                   
                   text_color=GRAY1,
                   text_font=FONT,
                   text_font_size='15pt',                   
                   text_font_style='normal'), 
             place="above")


# Add footing legend
p.add_layout(Title(text='Based on survey of 100 students conducted before and after pilot program (100% response rate on both surveys).', 
                   align="left",                   
                   text_color=GRAY6,
                   text_font=FONT,
                   text_font_size='10.5pt',
                   text_font_style='normal'), 
             "below")


# Add text annotations
# This is a weakness of bokeh, you cannot format a single word within a text. We have to do it separately.

# Before annotations
p.text(x=0, 
       y=0, 
       y_offset=-58,
       text=['BEFORE program, the'],       
       text_color=GRAY6,
       text_font=FONT,
       text_font_size='13pt')

p.text(x=0, 
       y=0,
       y_offset=-10,
       text=['majority of children (40%) \nfelt just Ok'],        
       text_color=ORANGE1,
       text_font=FONT,
       text_font_size='13pt')

p.text(x=0, 
       y=0,
       y_offset=-10,
       text=['\t' * 18 + 'about science.'],        
       text_color=GRAY6,
       text_font=FONT,
       text_font_size='13pt')

# After annotations
p.text(x=60, 
       y=0, 
       y_offset=-58,
       text=['AFTER program,'],       
       text_color=GRAY6,
       text_font=FONT,
       text_font_size='13pt')

p.text(x=60, 
       y=0,
       y_offset=-7,
       text=['\t' * 28 + 'more children \nwere Kind of interested (30%) \n& Excited (38%)'],       
       text_color=BLUE2,
       text_font=FONT,
       text_font_size='13pt')

p.text(x=60, 
       y=0,
       y_offset=-7,
       text=['\t' * 27 + 'about science.'], 
       text_color=GRAY6,
       text_font=FONT,
       text_font_size='13pt')


# Add top axis legend
p.text(x=-3, 
       y=3,
       y_offset=-30,
       text=['% total'], 
       text_color=GRAY8,
       text_font=FONT,
       text_font_size='13pt')


# Add header legends
p.text(x=-3, 
       y=3,
       y_offset=-60,
       text=['Bored' + '\t' * 16 + 'No great'],
       text_color=GRAY7,
       text_font=FONT,
       text_font_size='13pt',
       text_font_style='bold')

p.text(x=42, 
       y=3,
       y_offset=-60,
       text=['OK'],
       text_color=ORANGE1,
       text_font=FONT,
       text_font_size='13pt',
       text_font_style='bold')

p.text(x=58, 
       y=3,
       y_offset=-60,
       text=['Kind of interested' + '\t' * 15 + 'Excited'], 
       text_color=BLUE2,
       text_font=FONT,
       text_font_size='13pt',
       text_font_style='bold')

p.text(x=11, 
       y=3,
       y_offset=-60,
       text=[('|') + ('\t' * 26 + '|') + ('\t' * 16 + '|') +  ('\t' * 40 + '|')],       
       text_color=GRAY9,
       text_font=FONT,
       text_font_size='15pt', )


# Elements attributes

# Modify title attributes
p.title.background_fill_color = GRAY7
p.title.text_color = 'white'
p.title.text_font = FONT
p.title.text_font_size = '24pt'
p.title.text_font_style = 'normal'


# Modify X axis attributes
p.xaxis.bounds = (0, 100)
p.xaxis.fixed_location = 3
p.xaxis.axis_line_color = GRAY6
p.xaxis.formatter = PrintfTickFormatter(format='%0.0f %%')
p.xaxis.major_label_text_color = GRAY8
p.xaxis.major_label_text_font = FONT
p.xaxis.major_label_text_font_size = '12pt'
p.xaxis.major_tick_in = 0
p.xaxis.major_tick_line_color = GRAY8
p.xaxis.minor_tick_line_color = None
p.xgrid.grid_line_color = None


# Modify Y axis attributes
p.yaxis.axis_line_color = None
p.yaxis.major_label_standoff = -90
p.yaxis.major_label_text_color = GRAY7
p.yaxis.major_label_text_font = FONT
p.yaxis.major_label_text_font_size = '13pt'
p.yaxis.major_tick_line_color = None
p.ygrid.grid_line_color = None

# Convert the figure to png
p.background_fill_color = None
p.border_fill_color = None
p.outline_line_color = None

show(p)